In [1]:
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats
#Para Regressão Linear Múltipla (OLS, GLS e WLS) e Testes Estatísticos
import statsmodels.api as sm
import econtools
import econtools.metrics as mt
#Para Regressão em Painel
from linearmodels import PanelOLS, FirstDifferenceOLS, PooledOLS, RandomEffects

#Pacotes para gráficos (caso precise)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output
import gc

####################################### Criando as Funções ###############################################################

def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata (.dta) - NÃO COLOQUE A EXTENSÃO NA HORA DE NOMEAR O "NOME"!
    O arquivo deve estar na mesma pasta do arquivo de Python ou do notebook do jupyter.
    Deixe em branco para ler o arquivo mais recentemente adicionado à pasta.
    '''

    global df

    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #No meu caso específico:
    caminho_vinicius = f"{caminho}/datasets"

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        try:
            arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
        except:
            arquivo = max(glob.glob(f"{str(caminho_vinicius)}/*.dta"), key=os.path.getctime)
            df = pd.read_stata(arquivo)
            print(f"{arquivo}.dta foi lido com sucesso!")
            return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except:
            try:
                arquivo = f"{str(caminho_vinicius)}/{str(nome)}.dta"
                df = pd.read_stata(arquivo)
                print(f"{nome}.dta foi lido com sucesso!")
                return df
            except: #caso não tenha sido encontrado o arquivo com o nome inserido
                print('''
                Não foi possível achar o arquivo :(\n
                Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
                ''')

def Regressao_Multipla(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla, sendo, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares (OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x

    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.OLS(y,X)

    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(Resultado.summary())

    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}")
    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade.
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )

def Regressao_MQP(x, y, pesos, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados ponderados, ou seja,
    recomendada quando o erro é heteroscedástico E se sabe a função da constante. Ela é, por default, computada com um intercepto e com erros padrões não robustos.
    multiplicativa da variância do erro.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    pesos: 1/h, sendo h a constante multiplicativa da variância do erro;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.WLS(y,X, weights = pesos)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )
    
def Regressao_MQGF(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados generalizados factíveis, ou seja,
    recomendada quando o erro é heteroscedástico E NÃO se sabe a função da constante multiplicativa da variância do erro, sendo os pesos estimados
    regridindo o log dos quadrados dos resíduos sobre as variáveis explicativas. Os estimadores MQP são gerados com o peso estimado.
    Ela é, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #Regredindo os valores normalmente a fim de pegar os resíduos
    Regressao_Multipla(x,y, constante, robusta)
    clear_output()

    #Coletando o log dos quadrados dos resíduos
    Log_Res_Quad = np.log(Resíduos**2)

    #Regredindo Log_Res_Quad sobre as variáveis explicativas
    Regressao_Multipla(x,Log_Res_Quad, constante, robusta)
    clear_output()

    #Estimando os pesos
    Pesos = np.exp(Lista_ychapeu)

    #Fazendo uma Regressão MQP
    Regressao_MQP(x,y, 1/Pesos, constante, robusta)

def Teste_LM(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste LM e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)
    
    #Fazendo a regressão do modelo restrito e armazenando os resíduos
    Regressao_Multipla(df[ModeloRestrito], y)
    Resíduos_r = Resíduos

    #Fazendo a regressão dos resíduos sobre as variáveis independentes e armazenando o R2
    Regressao_Multipla(x, Resíduos_r)
    Ru = Resultado.rsquared

    #Calculando a estatística LM
    LM = Numero_de_Observações*Ru

    #Calculando o p-valor
    ##Calculando o P-valor de F
    P_valor = stats.chi2.sf(LM,GL_r)

    #Limpando a tela
    clear_output()

    #Printando o resultado
    if Nivel_de_Significância > P_valor:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")


def Teste_F(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    #para testar igualdade dos coeficientes, F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressao_Multipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressao_Multipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho à significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho à significância de {Nivel_de_Significância*100}%.")

def Teste_t_Dois_Coeficientes_Iguais(x, y, Coeficientes_Testados_para_serem_iguais, Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste t para verificar se dois coeficientes são iguais.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Coeficientes_Testados_para_serem_iguais: array com os valores dos coeficientes que querem ser testados;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    ##Fazendo a regressão do modelo irrestrito
    Regressao_Multipla(x, y)
    clear_output()

    #Fazendo o objeto de lista que será usado no teste
    Teste =[0]
    Num_de_Variaveis = 1

    for i in list(x):
        if i not in list(Coeficientes_Testados_para_serem_iguais):
            Teste.append(0)
        elif (Num_de_Variaveis % 2 == 0):
            Teste.append(-1)
        else:
            Teste.append(1)
            Num_de_Variaveis += 1

    Teste_t = Resultado.t_test(Teste)
    print(f"A estatística do teste é {Teste_t.tvalue}, o que resulta em um p-valor bilateral de {Teste_t.pvalue} e em um p-valor unilateral de {Teste_t.pvalue/2}.")

def Teste_Heteroscedasticidade_BP(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de Breusch-Pagan para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística = LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressao_Multipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resíduos**2

    #Realizando o teste F ou LM de Res_Quad sobre as variaveis dependentes para ver se há correlação
    if Estatística == "LM":
        Teste_LM(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def Teste_Heteroscedasticidade_White(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de White (modificado por Wooldridge) para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística: LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressao_Multipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resultado.resid**2

    #Calculando o quadrado dos valores previstos
    Previstos = Lista_ychapeu
    Previstos2 = Previstos**2

    #Criando um dataframe pra armazenar esses valores
    dfy_y2 = pd.DataFrame({"y":Previstos,"y2":Previstos2})
    y_y2 = dfy_y2[['y','y2']]

    #Realizando o teste F ou LM de Res_Quad sobre y e y^2
    if Estatística == "LM":
        Teste_LM(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def RESET(x, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste RESET para verificar a adequação das formas funcionais.
    Ho: o modelo está bem especificado.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    #Fazendo uma regressão múltipla e limpando a tela
    Regressao_Multipla(x, y, constante)
    clear_output()

    #Verificando o tipo da covariância selecionada
    if robusta == "N":
        tipo = 'nonrobust'
    else:
        tipo = 'HC1'

    Teste = sm.stats.diagnostic.linear_reset(Resultado, power = 2, use_f = False, cov_type = tipo)
    
    if Teste.pvalue < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (o modelo está MAL especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, maior que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (o modelo NÃO está MAL especificado)"""
        )

def Teste_J_Davidson_MacKinnon(x1,x2, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste J para verificar qual o modelo mais adequado (dentre os dois colocados).
    Ho: o modelo 1 é preferível (ver o p-valor do último coeficiente).

    x1: lista ou array com os valores das variáveis independentes do primeiro modelo;
    x2: lista ou array com os valores das variáveis independentes do segundo modelo;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    #Fazendo a regressão do segundo modelo
    Regressao_Multipla(x2, y, constante, robusta)
    clear_output()

    #Criando um novo dataframe e adicionando os valores previstos do modelo 2 à x
    Valores_Previstos_2 = pd.DataFrame({'Previsão M1':Lista_ychapeu})
    x = pd.concat([x1, Valores_Previstos_2], axis=1, sort=False)

    #Fazendo a regressão do primeiro modelo sobre x
    Regressao_Multipla(x, y, constante, robusta)
    clear_output()

    #Pegando o p-valor do teste
    P_valor = Resultado.pvalues[-1]

    if P_valor < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (ou seja, o modelo 2 ({list(x2)}) é mais bem especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (ou seja, o modelo 1 ({list(x1)}) é mais bem especificado)."""
        )

######### Funções de Dados em Painel #########
def Arrumar_Painel():
    '''
    Função que transforma o painel num formato que o PanelOLS consegue ler (index multinível e coluna do tipo categoria para os anos)
    '''
    global df

    # pedir a coluna com os indivíduos; se o nome for inválido, sair da função.
    coluna_individuos = str(input('Qual o rótulo da coluna de indivíduos/observações?\n'))
    if coluna_individuos not in df.columns:
        print("Coluna de indivíduos não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None
    
    # pedir a coluna com os períodos de tempo; se o valor for inválido, sair da função.
    coluna_tempo = str(input('Qual o rótulo da coluna de tempo?\n'))
    if coluna_tempo not in df.columns:
        print("Coluna de tempo não está no dataframe. Insira uma coluna válida e tente novamente!")
        return None

    ## arrumando o painel
    periodos = pd.Categorical(df[coluna_tempo])
    df = df.set_index([coluna_individuos,coluna_tempo])
    df[coluna_tempo] = periodos
    return df

def Reg_Painel_Primeiras_Diferenças (x,y, robusta = "N"):
    '''
    Função que calcula uma regressão de primeiras diferenças SEM um intercepto, sendo, por default, computada com erros padrões não robustos.
    Para calcular a regressão com um intercepto, ver o notebook "Cap 13 e 14".
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    Modelo = FirstDifferenceOLS(y, x)
    Resultado = Modelo.fit()
    print(Resultado)

def Reg_Painel_Efeitos_Fixos(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões não robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PanelOLS(y,X, entity_effects=True)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    print(Resultado)

def Reg_Painel_MQO_Agrupado(x, y, constante = "S", robusta = "S"):
    '''
    Função que calcula uma regressão por MQO agrupado, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = PooledOLS(y,X)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')
    print(Resultado)

def Reg_Painel_Efeitos_Aleatórios(x, y, constante = "S"):
    '''
    Função que calcula uma regressão de efeitos fixos, sendo, por default, computada com um intercepto e com erros padrões  robustos.
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão robustos.
    '''
    global df, Resultado
    
    # formando o vetor de variáveis independentes
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    
    #Criando o Modelo
    Modelo = RandomEffects(y,X)
    Resultado = Modelo.fit()
    print(Resultado)

## Exercício C1

In [2]:
coletar_dados('fertil1')

fertil1.dta foi lido com sucesso!


,year,educ,meduc,feduc,age,kids,black,east,northcen,west,...,y80,y82,y84,agesq,y74educ,y76educ,y78educ,y80educ,y82educ,y84educ
0,72,12,8,8,48,4,0,0,1,0,...,0,0,0,2304,0,0,0,0,0,0
1,72,17,8,18,46,3,0,0,0,0,...,0,0,0,2116,0,0,0,0,0,0
2,72,12,7,8,53,2,0,0,1,0,...,0,0,0,2809,0,0,0,0,0,0
3,72,12,12,10,42,2,0,0,1,0,...,0,0,0,1764,0,0,0,0,0,0
4,72,12,3,8,51,2,0,0,0,0,...,0,0,0,2601,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,84,12,12,12,41,1,0,0,0,1,...,0,0,1,1681,0,0,0,0,0,12
1125,84,19,10,15,42,0,0,0,0,1,...,0,0,1,1764,0,0,0,0,0,19
1126,84,15,12,10,35,3,0,0,0,1,...,0,0,1,1225,0,0,0,0,0,15
1127,84,12,12,12,46,2,0,0,1,0,...,0,0,1,2116,0,0,0,0,0,12


In [3]:
df.columns

Index(['year', 'educ', 'meduc', 'feduc', 'age', 'kids', 'black', 'east',
       'northcen', 'west', 'farm', 'othrural', 'town', 'smcity', 'y74', 'y76',
       'y78', 'y80', 'y82', 'y84', 'agesq', 'y74educ', 'y76educ', 'y78educ',
       'y80educ', 'y82educ', 'y84educ'],
      dtype='object')

In [4]:
## Fazendo o modelo de regressão
x = df[['educ','age','agesq','black','east','northcen','west','farm','othrural','town','smcity','y74','y76','y78','y80','y82','y84']]
y = df['kids']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                   kids   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     9.723
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           2.42e-24
Time:                        18:01:12   Log-Likelihood:                -2091.2
No. Observations:                1129   AIC:                             4218.
Df Residuals:                    1111   BIC:                             4309.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.7425      3.052     -2.537      0.0

In [5]:
## Fazendo o Teste F retirando variáveis ambientais
restricoes = df[['farm','othrural','town','smcity']]
Teste_F(x,y,restricoes)

O valor de F é 1.159 e seu p-valor é 0.327458. Portanto, não se rejeita Ho à significância de 5.0%.


In [6]:
## Fazendo o teste F retirando variáveis regionais
restricoes = df[['east','northcen','west']]
Teste_F(x,y,restricoes)

O valor de F é 3.012 e seu p-valor é 0.029258. Portanto, rejeita-se Ho à significância de 5.0%.


In [7]:
## Fazendo o modelo de regressão
x = df[['educ','age','agesq','black','east','northcen','west','farm','othrural','town','smcity','y74','y76','y78','y80','y82','y84']]
y = df['kids']
Regressao_Multipla(x,y)

## Coletando os Residuos quadráticos e aplicando a regressão
clear_output()
u = Resíduos**2
dummies_temporais = df[['y74','y76','y78','y80','y82','y84']]
Regressao_Multipla(dummies_temporais,u)

                            OLS Regression Results                            
Dep. Variable:                   kids   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     2.905
Date:                Sun, 07 Mar 2021   Prob (F-statistic):            0.00816
Time:                        18:01:13   Log-Likelihood:                -2940.1
No. Observations:                1129   AIC:                             5894.
Df Residuals:                    1122   BIC:                             5929.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.2873      0.263     12.512      0.0

In [8]:
## Adicionando os termos de interação
x = df[['educ','age','agesq','black','east','northcen','west','farm','othrural','town','smcity','y74','y76','y78','y80','y82','y84','y74educ', 'y76educ', 'y78educ','y80educ', 'y82educ', 'y84educ']]
y = df['kids']

Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                   kids   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     7.593
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           3.41e-23
Time:                        18:01:13   Log-Likelihood:                -2086.7
No. Observations:                1129   AIC:                             4221.
Df Residuals:                    1105   BIC:                             4342.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -8.4773      3.126     -2.712      0.0

In [9]:
## Fazendo o teste F retirando os termos de interação
restricoes = df[['y74educ', 'y76educ', 'y78educ','y80educ', 'y82educ', 'y84educ']]
Teste_F(x,y,restricoes)

O valor de F é 1.484 e seu p-valor é 0.1803366. Portanto, não se rejeita Ho à significância de 5.0%.


## Exercício C2

In [10]:
coletar_dados('CPS78_85')

CPS78_85.dta foi lido com sucesso!


,educ,south,nonwhite,female,married,exper,expersq,union,lwage,age,year,y85,y85fem,y85educ,y85union
0,12,0,0,0,0,8,64,0,1.2150,25,78,0,0,0,0
1,12,0,0,1,1,30,900,1,1.6094,47,78,0,0,0,0
2,6,0,0,0,1,38,1444,1,2.1401,49,78,0,0,0,0
3,12,0,0,0,1,19,361,1,2.0732,36,78,0,0,0,0
4,12,0,0,0,1,11,121,0,1.6490,28,78,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079,12,0,1,1,1,18,324,0,2.1552,36,85,1,1,12,0
1080,14,0,0,1,0,13,169,0,2.9178,33,85,1,1,14,0
1081,18,0,0,0,1,8,64,0,3.1001,32,85,1,0,18,0
1082,12,0,1,0,0,14,196,1,2.7887,32,85,1,0,12,1


In [11]:
df.columns

Index(['educ', 'south', 'nonwhite', 'female', 'married', 'exper', 'expersq',
       'union', 'lwage', 'age', 'year', 'y85', 'y85fem', 'y85educ',
       'y85union'],
      dtype='object')

In [12]:
## Fazendo o modelo de regressão
x = df[['y85','educ','y85educ','exper','expersq','union','female','y85fem']]
y = df['lwage']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     99.80
Date:                Sun, 07 Mar 2021   Prob (F-statistic):          4.46e-124
Time:                        18:01:13   Log-Likelihood:                -574.24
No. Observations:                1084   AIC:                             1166.
Df Residuals:                    1075   BIC:                             1211.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4589      0.093      4.911      0.0

In [13]:
## Fazendo o modelo de regressão
df['educ_12'] = df['educ'] - 12
df['y85educ_12'] = df['y85']*df['educ_12']

x = df[['y85','educ_12','y85educ_12','exper','expersq','union','female','y85fem']]
y = df['lwage']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     99.80
Date:                Sun, 07 Mar 2021   Prob (F-statistic):          4.46e-124
Time:                        18:01:14   Log-Likelihood:                -574.24
No. Observations:                1084   AIC:                             1166.
Df Residuals:                    1075   BIC:                             1211.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3556      0.039     35.068      0.0

In [14]:
## Criando a coluna de salário real
df['lrwage'] = np.where(df['y85'] == 0, df['lwage'], df['lwage']-np.log(1.65))

## Fazendo o modelo de regressão
x = df[['y85','educ','y85educ','exper','expersq','union','female','y85fem']]
y = df['lrwage']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                 lrwage   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     74.35
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           1.87e-97
Time:                        18:01:14   Log-Likelihood:                -574.24
No. Observations:                1084   AIC:                             1166.
Df Residuals:                    1075   BIC:                             1211.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4589      0.093      4.911      0.0

In [15]:
part_union_78 = len(df.loc[(df['y85'] == 0) & (df['union'] == 1)])/len(df.loc[df['y85'] == 0])
part_union_85 = len(df.loc[(df['y85'] == 1) & (df['union'] == 1)])/len(df.loc[df['y85'] == 1])

print(f"O percentual de sindicalizados em 1978 era {round(part_union_78*100,2)}%, enquanto em 1985 foi {round(part_union_85*100,2)}%.\nA mudança foi de {round((part_union_85-part_union_78)*100,2)}%.")

O percentual de sindicalizados em 1978 era 30.55%, enquanto em 1985 foi 17.98%.
A mudança foi de -12.57%.


In [16]:
## Fazendo o modelo com o termo de interação pra captar as mudanças do efeito de union ao longo do período
x = df[['y85','educ','y85educ','exper','expersq','union','female','y85fem','y85union']]
y = df['lwage']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     88.63
Date:                Sun, 07 Mar 2021   Prob (F-statistic):          4.61e-123
Time:                        18:01:14   Log-Likelihood:                -574.24
No. Observations:                1084   AIC:                             1168.
Df Residuals:                    1074   BIC:                             1218.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4588      0.095      4.851      0.0

## Exercício C3

In [17]:
coletar_dados('kielmc')

kielmc.dta foi lido com sucesso!


,year,age,agesq,nbh,cbd,intst,lintst,price,rooms,area,...,lprice,y81,larea,lland,y81ldist,lintstsq,nearinc,y81nrinc,rprice,lrprice
0,1978,48,2304.0,4,3000.0,1000.0,6.9078,60000.0,7,1660,...,11.002100,0,7.414573,8.429017,0.000000,47.717705,1,0,60000.000000,11.002100
1,1978,83,6889.0,4,4000.0,1000.0,6.9078,40000.0,6,2612,...,10.596635,0,7.867871,9.032409,0.000000,47.717705,1,0,40000.000000,10.596635
2,1978,58,3364.0,4,4000.0,1000.0,6.9078,34000.0,6,1144,...,10.434115,0,7.042286,8.517193,0.000000,47.717705,1,0,34000.000000,10.434115
3,1978,11,121.0,4,4000.0,1000.0,6.9078,63900.0,5,1136,...,11.065075,0,7.035269,9.210340,0.000000,47.717705,1,0,63900.000000,11.065075
4,1978,48,2304.0,4,4000.0,2000.0,7.6009,44000.0,5,1868,...,10.691945,0,7.532624,9.210340,0.000000,57.773682,1,0,44000.000000,10.691945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,1981,25,625.0,0,10000.0,10000.0,9.2103,60000.0,6,1388,...,11.002100,1,7.235619,8.922658,9.735069,84.829636,0,0,46082.949219,10.738198
317,1981,0,0.0,6,24000.0,26000.0,10.1660,144500.0,7,3140,...,11.881035,1,8.051978,10.795219,10.126630,103.347565,0,0,110983.101562,11.617133
318,1981,20,400.0,0,15000.0,15000.0,9.6158,97000.0,6,1296,...,11.482467,1,7.167038,10.711458,9.966462,92.463608,0,0,74500.765625,11.218565
319,1981,19,361.0,0,14000.0,15000.0,9.6158,95000.0,5,1148,...,11.461632,1,7.045776,10.696073,9.928180,92.463608,0,0,72964.671875,11.197731


In [18]:
df.columns

Index(['year', 'age', 'agesq', 'nbh', 'cbd', 'intst', 'lintst', 'price',
       'rooms', 'area', 'land', 'baths', 'dist', 'ldist', 'wind', 'lprice',
       'y81', 'larea', 'lland', 'y81ldist', 'lintstsq', 'nearinc', 'y81nrinc',
       'rprice', 'lrprice'],
      dtype='object')

In [19]:
## Fazendo o modelo 
x = df[['y81','ldist','y81ldist']]
y = df['lprice']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.390
Method:                 Least Squares   F-statistic:                     69.22
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           1.87e-34
Time:                        18:01:14   Log-Likelihood:                -109.24
No. Observations:                 321   AIC:                             226.5
Df Residuals:                     317   BIC:                             241.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.0585      0.508     15.850      0.0

In [20]:
## Fazendo o modelo 
x = df[['y81','ldist','y81ldist','age','agesq','rooms','baths','lintstsq','lland','larea']]
y = df['lprice']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     115.1
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           4.30e-98
Time:                        18:01:14   Log-Likelihood:                 58.724
No. Observations:                 321   AIC:                            -95.45
Df Residuals:                     310   BIC:                            -53.96
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.3095      0.531     13.768      0.0

## Exercício C4

In [21]:
coletar_dados('injury')

injury.dta foi lido com sucesso!


,durat,afchnge,highearn,male,married,hosp,indust,injtype,age,prewage,...,head,neck,upextr,trunk,lowback,lowextr,occdis,manuf,construc,highlpre
0,1.0,1,1,1.0,0.0,1,3.0,1,26.0,404.950012,...,1,0,0,0,0,0,0,0.0,0.0,6.003764
1,1.0,1,1,1.0,1.0,0,3.0,1,31.0,643.825012,...,1,0,0,0,0,0,0,0.0,0.0,6.467427
2,84.0,1,1,1.0,1.0,1,3.0,1,37.0,398.125000,...,1,0,0,0,0,0,0,0.0,0.0,5.986766
3,4.0,1,1,1.0,1.0,1,3.0,1,31.0,527.799988,...,1,0,0,0,0,0,0,0.0,0.0,6.268717
4,1.0,1,1,1.0,1.0,0,3.0,1,23.0,528.937500,...,1,0,0,0,0,0,0,0.0,0.0,6.270870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7145,1.0,0,0,1.0,1.0,0,3.0,7,23.0,284.299988,...,0,0,0,0,0,0,1,0.0,0.0,0.000000
7146,18.0,0,0,0.0,0.0,0,3.0,7,31.0,304.769592,...,0,0,0,0,0,0,1,0.0,0.0,0.000000
7147,14.0,0,0,0.0,0.0,0,3.0,7,66.0,255.869995,...,0,0,0,0,0,0,1,0.0,0.0,0.000000
7148,76.0,0,0,0.0,NaN,0,3.0,7,39.0,274.065186,...,0,0,0,0,0,0,1,0.0,0.0,0.000000


In [22]:
df.columns

Index(['durat', 'afchnge', 'highearn', 'male', 'married', 'hosp', 'indust',
       'injtype', 'age', 'prewage', 'totmed', 'injdes', 'benefit', 'ky', 'mi',
       'ldurat', 'afhigh', 'lprewage', 'lage', 'ltotmed', 'head', 'neck',
       'upextr', 'trunk', 'lowback', 'lowextr', 'occdis', 'manuf', 'construc',
       'highlpre'],
      dtype='object')

In [23]:
## Tirando dados nulos
df.dropna(subset = ['male','married','head','neck','upextr','trunk', 'lowback', 'lowextr', 'occdis', 'manuf', 'construc'],inplace=True, axis=0)

In [24]:
## Usando apenas os dados de Kentucky
df_ky = df.loc[df['ky'] == 1]

## Criando o modelo
x = df_ky[['afchnge','highearn','afhigh','male','married','head','neck','upextr','trunk', 'lowback', 'lowextr', 'occdis', 'manuf', 'construc']]
y = df_ky['ldurat']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                 ldurat   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     16.37
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           4.81e-40
Time:                        18:01:15   Log-Likelihood:                -8778.2
No. Observations:                5349   AIC:                         1.759e+04
Df Residuals:                    5334   BIC:                         1.769e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2459      0.106     11.735      0.0

In [25]:
## Usando apenas os dados de Michigan
df_mi = df.loc[df['mi'] == 1]

## Criando o modelo da equação (13.2)
x = df_mi[['afchnge','highearn','afhigh']]
y = df_mi['ldurat']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                 ldurat   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     5.039
Date:                Sun, 07 Mar 2021   Prob (F-statistic):            0.00178
Time:                        18:01:15   Log-Likelihood:                -2556.1
No. Observations:                1475   AIC:                             5120.
Df Residuals:                    1471   BIC:                             5141.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4317      0.058     24.827      0.0

## Exercício C5

In [26]:
coletar_dados('rental')

rental.dta foi lido com sucesso!


,city,year,pop,enroll,rent,rnthsg,tothsg,avginc,lenroll,lpop,...,lavginc,clenroll,clpop,clrent,cltothsg,clrnthsg,clavginc,pctstu,cpctstu,y90
0,1,80,75211.0,15303.0,197,13475.0,26167.0,11537.0,9.635804,11.228053,...,9.353314,NaN,NaN,NaN,NaN,NaN,NaN,20.346758,NaN,0
1,1,90,77759.0,18017.0,342,15660.0,29467.0,19568.0,9.799071,11.261370,...,9.881651,-15293.201172,0.033317,0.551607,0.118772,0.150273,0.528337,23.170309,2.823551,1
2,2,80,106743.0,22462.0,323,14580.0,37277.0,19841.0,10.019580,11.578179,...,9.895506,NaN,NaN,NaN,NaN,NaN,NaN,21.043066,NaN,0
3,2,90,141865.0,29769.0,496,26895.0,55540.0,31885.0,10.301223,11.862631,...,10.369891,-22451.699219,0.284451,0.428924,0.398727,0.612289,0.474385,20.984034,-0.059032,1
4,3,80,36608.0,11847.0,216,7026.0,13482.0,11455.0,9.379830,10.508022,...,9.346182,NaN,NaN,NaN,NaN,NaN,NaN,32.361778,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,62,90,56856.0,10640.0,352,8980.0,21118.0,24735.0,9.272376,10.948277,...,10.115974,-9600.727539,0.098765,0.447531,0.140006,0.225234,0.537041,18.713943,0.057011,1
124,63,80,48347.0,9051.0,220,8224.0,18085.0,13458.0,9.110631,10.786160,...,9.507329,NaN,NaN,NaN,NaN,NaN,NaN,18.720913,NaN,0
125,63,90,51003.0,9961.0,344,10073.0,19970.0,21947.0,9.206432,10.839640,...,9.996386,-9041.793945,0.053480,0.447014,0.099148,0.202803,0.489057,19.530224,0.809311,1
126,64,80,170616.0,37475.0,243,34108.0,66451.0,16510.0,10.531429,12.047171,...,9.711721,NaN,NaN,NaN,NaN,NaN,NaN,21.964529,NaN,0


In [27]:
df.columns

Index(['city', 'year', 'pop', 'enroll', 'rent', 'rnthsg', 'tothsg', 'avginc',
       'lenroll', 'lpop', 'lrent', 'ltothsg', 'lrnthsg', 'lavginc', 'clenroll',
       'clpop', 'clrent', 'cltothsg', 'clrnthsg', 'clavginc', 'pctstu',
       'cpctstu', 'y90'],
      dtype='object')

In [28]:
## Ajustando o formato do dataset
Arrumar_Painel()

Coluna de indivíduos não está no dataframe. Insira uma coluna válida e tente novamente!


In [29]:
## Criando o modelo usando MQO Agrupado

x = df[['y90','lpop','lavginc','pctstu']]
y = df['lrent']
Reg_Painel_MQO_Agrupado(x,y)

ValueError: Series can only be used with a 2-level MultiIndex

In [80]:
## Criando o modelo usando primeiras diferenças
df_90 = df.loc[df['y90'] == 1]

x = df_90[['y90','clpop','clavginc','cpctstu']]
y = df_90['clrent']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                 clrent   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     9.510
Date:                Wed, 03 Mar 2021   Prob (F-statistic):           3.14e-05
Time:                        20:23:09   Log-Likelihood:                 65.272
No. Observations:                  64   AIC:                            -122.5
Df Residuals:                      60   BIC:                            -113.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
y90            0.3855      0.037     10.469      0.0

In [81]:
## Vendo os erros padrão dos coeficientes
Resultado.bse

y90         0.036824
clpop       0.088343
clavginc    0.066477
cpctstu     0.004132
dtype: float32

In [82]:
## Criando o modelo usando primeiras diferenças e erros robustos
df_90 = df.loc[df['y90'] == 1]

x = df_90[['y90','clpop','clavginc','cpctstu']]
y = df_90['clrent']
Regressao_Multipla(x,y,robusta='S')

                            OLS Regression Results                            
Dep. Variable:                 clrent   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     11.30
Date:                Wed, 03 Mar 2021   Prob (F-statistic):           5.64e-06
Time:                        20:23:11   Log-Likelihood:                 65.272
No. Observations:                  64   AIC:                            -122.5
Df Residuals:                      60   BIC:                            -113.9
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
y90            0.3855      0.049      7.913      0.0

In [83]:
Resultado.bse

y90         0.048719
clpop       0.069680
clavginc    0.089310
cpctstu     0.002936
dtype: float32

## Exercicio C1O

In [30]:
coletar_dados('jtrain')

jtrain.dta foi lido com sucesso!


,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,1988,419483.0,108.0,11500000.0,14810.0,25.0,NaN,0.0,1,0,...,0,0.223144,0,-0.208218,0.044453,9.603058,0.059321,0.0,0.000000,0.000000
467,1989,419483.0,129.0,12000000.0,14227.0,30.0,NaN,20.0,1,0,...,0,0.182322,0,0.177681,0.042559,9.562897,-0.040161,0.0,3.100775,1.411176
468,1987,419486.0,80.0,7000000.0,16000.0,NaN,NaN,0.0,0,0,...,0,NaN,0,NaN,NaN,9.680344,NaN,NaN,NaN,NaN
469,1988,419486.0,90.0,8500000.0,17000.0,NaN,NaN,0.0,0,0,...,0,NaN,0,0.117783,0.194157,9.740969,0.060625,0.0,0.000000,0.000000


In [31]:
df.columns

Index(['year', 'fcode', 'employ', 'sales', 'avgsal', 'scrap', 'rework',
       'tothrs', 'union', 'grant', 'd89', 'd88', 'totrain', 'hrsemp', 'lscrap',
       'lemploy', 'lsales', 'lrework', 'lhrsemp', 'lscrap_1', 'grant_1',
       'clscrap', 'cgrant', 'clemploy', 'clsales', 'lavgsal', 'clavgsal',
       'cgrant_1', 'chrsemp', 'clhrsemp'],
      dtype='object')

In [54]:
# Capturando apenas os dados de 1988 e 1989
df_88_89 = df.loc[df['year'] != 1987]

# contando o número de observações nulas
df_88_89[['d89','cgrant','cgrant_1','clemploy','chrsemp']].isna().sum()

d89          0
cgrant       0
cgrant_1     0
clemploy    24
chrsemp     63
dtype: int64

In [58]:
## Arrumando o dataframe para primeiras diferenças
df_88_89 = df.loc[df['year'] != 1987]
df_88_89 = df_88_89.dropna(subset = ['d89','cgrant','cgrant_1','clemploy','chrsemp'], axis = 0)

In [56]:
## Fazendo a regressão de 1as diferenças
x = df_88_89[['d89','cgrant','cgrant_1','clemploy']]
y = df_88_89['chrsemp']
Regressao_Multipla(x,y)

                            OLS Regression Results                            
Dep. Variable:                chrsemp   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     55.80
Date:                Sun, 07 Mar 2021   Prob (F-statistic):           1.92e-33
Time:                        18:21:09   Log-Likelihood:                -1098.3
No. Observations:                 251   AIC:                             2207.
Df Residuals:                     246   BIC:                             2224.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7400      1.942     -0.381      0.7

In [60]:
## Contando quantas firmas tem dados completos
print(f"O número de códigos únicos é: {df_88_89['fcode'].nunique()}")

O número de códigos únicos é: 127


In [47]:
## O coeficiente de grant indica que a concessão do grant aumenta o numero de horas de treinamento de um trabalhador em 32,6h por ano.

In [48]:
## Os efeitos de grant_1 sobre o número de horas de treinamento já podem ter sido inteiramente consumidos no ano anterior, uma vez que a firma pode optar por consumir esses recursos de forma mais rápida

## Exercício C13

In [270]:
coletar_dados('wagepan')

wagepan.dta foi lido com sucesso!


,nr,year,agric,black,bus,construc,ent,exper,fin,hisp,...,union,lwage,d81,d82,d83,d84,d85,d86,d87,expersq
0,13,1980,0,0,1,0,0,1,0,0,...,0,1.197540,0,0,0,0,0,0,0,1
1,13,1981,0,0,0,0,0,2,0,0,...,1,1.853060,1,0,0,0,0,0,0,4
2,13,1982,0,0,1,0,0,3,0,0,...,0,1.344462,0,1,0,0,0,0,0,9
3,13,1983,0,0,1,0,0,4,0,0,...,0,1.433213,0,0,1,0,0,0,0,16
4,13,1984,0,0,0,0,0,5,0,0,...,0,1.568125,0,0,0,1,0,0,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4355,12548,1983,0,0,0,1,0,8,0,0,...,0,1.591879,0,0,1,0,0,0,0,64
4356,12548,1984,0,0,0,1,0,9,0,0,...,1,1.212543,0,0,0,1,0,0,0,81
4357,12548,1985,0,0,0,1,0,10,0,0,...,0,1.765962,0,0,0,0,1,0,0,100
4358,12548,1986,0,0,0,0,0,11,0,0,...,1,1.745894,0,0,0,0,0,1,0,121


In [271]:
df.columns

Index(['nr', 'year', 'agric', 'black', 'bus', 'construc', 'ent', 'exper',
       'fin', 'hisp', 'poorhlth', 'hours', 'manuf', 'married', 'min',
       'nrthcen', 'nrtheast', 'occ1', 'occ2', 'occ3', 'occ4', 'occ5', 'occ6',
       'occ7', 'occ8', 'occ9', 'per', 'pro', 'pub', 'rur', 'south', 'educ',
       'tra', 'trad', 'union', 'lwage', 'd81', 'd82', 'd83', 'd84', 'd85',
       'd86', 'd87', 'expersq'],
      dtype='object')

In [272]:
## criando as var. de interação
df['d81educ'] = df['d81']*df['educ']
df['d82educ'] = df['d82']*df['educ']
df['d83educ'] = df['d83']*df['educ']
df['d84educ'] = df['d84']*df['educ']
df['d85educ'] = df['d85']*df['educ']
df['d86educ'] = df['d86']*df['educ']
df['d87educ'] = df['d87']*df['educ']

In [273]:
## Usando PD, poderíamos estimar todos os parametros com a excecao do intercepto de 1980 e educ

In [274]:
## Arrumando o  painel para PD
Arrumar_Painel()

agric  black  bus  construc  ent  exper  fin  hisp  poorhlth  \
nr    year                                                                 
13    1980      0      0    1         0    0      1    0     0         0   
      1981      0      0    0         0    0      2    0     0         0   
      1982      0      0    1         0    0      3    0     0         0   
      1983      0      0    1         0    0      4    0     0         0   
      1984      0      0    0         0    0      5    0     0         0   
...           ...    ...  ...       ...  ...    ...  ...   ...       ...   
12548 1983      0      0    0         1    0      8    0     0         0   
      1984      0      0    0         1    0      9    0     0         0   
      1985      0      0    0         1    0     10    0     0         0   
      1986      0      0    0         0    0     11    0     0         0   
      1987      0      0    0         0    0     12    0     0         0   

            hours  ...  d87  expersq  d81educ  d82educ  d83educ  d84educ  \
nr    year         ...                                                     
13    1980   2672  ...    0        1        0        0        0        0   
      1981   2320  ...    0        4       14        0        0        0   
      1982   2940  ...    0        9        0       14        0        0   
      1983   2960  ...    0       16        0        0       14        0   
      1984   3071  ...    0       25        0        0        0       14   
...           ...  ...  ...      ...      ...      ...      ...      ...   
12548 1983   2080  ...    0       64        0        0        9        0   
      1984   2080  ...    0       81        0        0        0        9   
      1985   2080  ...    0      100        0        0        0        0   
      1986   2080  ...    0      121        0        0        0        0   
      1987   3380  ...    1      144        0        0        0        0   

            d85educ  d86educ  d87educ  year  
nr    year                                   
13    1980        0        0        0  1980  
      1981        0        0        0  1981  
      1982        0        0        0  1982  
      1983        0        0        0  1983  
      1984        0        0        0  1984  
...             ...      ...      ...   ...  
12548 1983        0        0        0  1983  
      1984        0        0        0  1984  
      1985        9        0        0  1985  
      1986        0        9        0  1986  
      1987        0        0        9  1987  

[4360 rows x 50 columns]

In [275]:
def Reg_Painel_Primeiras_Diferenças (x,y, robusta = "N"):
    '''
    Função que calcula uma regressão de primeiras diferenças SEM um intercepto, sendo, por default, computada com erros padrões não robustos.
    Para calcular a regressão com um intercepto, ver o notebook "Cap 13 e 14".
    **IMPORTANTE: para o painel estar arrumado, os dados devem estar multi-indexados por indíviduo e por tempo, nesta ordem.
    Caso contrário, transformar o dataframe usando a função 'Arrumar Painel'
    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''
    global df, Resultado

    Modelo = FirstDifferenceOLS(y, x)

    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'robust')

    print(Resultado)

In [276]:
## fazendo a regressão de primeiras diferenças (modelo irrestrito)
x = df[['year','d81educ', 'd82educ', 'd83educ', 'd84educ','d85educ', 'd86educ', 'd87educ','union']]
y= df['lwage']
Reg_Painel_Primeiras_Diferenças(x,y, robusta='S')

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  lwage   R-squared:                        0.0269
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.2843
No. Observations:                3815   R-squared (Within):               0.1678
Date:                Tue, Mar 09 2021   R-squared (Overall):              0.2792
Time:                        11:28:17   Log-likelihood                   -2305.2
Cov. Estimator:                Robust                                           
                                        F-statistic:                      7.0037
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(15,3800)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             6.4397
                            

In [255]:
## Fazendo um Teste de Wald robusto
restricoes = 'd81educ = d82educ = d83educ = d84educ = d85educ = d86educ = d87educ = 0'
Resultado.wald_test(formula=restricoes)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 2.2231
P-value: 0.9465
Distributed: chi2(7)
WaldTestStatistic, id: 0x7f8014ea6d50

In [258]:
for column in df[['d81','d82','d83','d84','d85','d86','d87','d81educ', 'd82educ', 'd83educ', 'd84educ','d85educ', 'd86educ', 'd87educ','union','lwage']].columns:
    df[f'c{column}'] = df[column].diff()

In [259]:
df_pd = df.loc[df['year'] != 1980]
x = df_pd[['cd81','cd82','cd83','cd84','cd85','cd86','cd87','cd81educ', 'cd82educ', 'cd83educ', 'cd84educ','cd85educ', 'cd86educ', 'cd87educ','cunion']]
y= df_pd['clwage']

Regressao_Multipla(x,y, robusta = 'S')

                            OLS Regression Results                            
Dep. Variable:                 clwage   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.440
Date:                Tue, 09 Mar 2021   Prob (F-statistic):           9.03e-14
Time:                        11:16:57   Log-Likelihood:                -2305.2
No. Observations:                3815   AIC:                             4640.
Df Residuals:                    3800   BIC:                             4734.
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0114      0.046      0.245      0.8

In [260]:
h0 = '(cd81educ = 0, cd82educ = 0, cd83educ = 0, cd84educ = 0, cd85educ = 0, cd86educ = 0, cd87educ = 0)'
ho = 'cd81educ = cd82educ = cd83educ = cd84educ = cd85educ = cd86educ = cd87educ = 0'
Resultado.wald_test(ho)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.31758259]]), p=0.9464166620135541, df_denom=3.8e+03, df_num=7>

In [263]:
def Teste_F(H0, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F com base nas restrições de H0
    H0 deve estar na forma B1 = B2 =...= Valor que deseja ser testado (0 na maioria das vezes)
    '''
    global Resultado
    ## A função utiliza o método wald_test dos resultados das regressões
    # Para modelos de painel - cujo método usa a estatística LM -, devemos especificar o parâmetro 'formula', o que não ocorre com cortes transversais
    try:
        teste = 'LM'
        est = Resultado.wald_test(formula=H0).stat
        p = Resultado.wald_test(formula=H0).pval
    except:
        teste = 'F'
        est = float(str(Resultado.wald_test(H0))[19:29])
        p = float(str(Resultado.wald_test(H0))[36:47])

    if Nivel_de_Significância > p:
        print(f"O valor de {teste} é {round(est,6)} e seu p-valor é {round(p,7)}. Portanto, rejeita-se Ho à significância de {Nivel_de_Significância*100}%, ou seja, as variáveis são conjuntamente significantes.")
    else:
        print(f"O valor de {teste} é {round(est,6)} e seu p-valor é {round(p,7)}. Portanto, não se rejeita Ho à significância de {Nivel_de_Significância*100}%, ou seja, as variáveis NÃO são conjuntamente significantes.")

In [264]:
Teste_F(ho)

O valor de F é 0.317583 e seu p-valor é 0.9464167. Portanto, não se rejeita Ho à significância de 5.0%, ou seja, as variáveis NÃO são conjuntamente significantes.


### Fazendo um teste robusto para heteroscedasticidade

In [265]:
coletar_dados('gpa3')

gpa3.dta foi lido com sucesso!


,term,sat,tothrs,cumgpa,season,frstsem,crsgpa,verbmath,trmgpa,hssize,...,female,black,white,ctrmgpa,ctothrs,ccrsgpa,ccrspop,cseason,hsperc,football
0,1,920,31,2.25,0,0,2.6464,0.48387,1.50,10,...,1,0,0,NaN,NaN,NaN,NaN,NaN,40.000000,0
1,2,920,43,2.04,1,0,2.5077,0.48387,2.25,10,...,1,0,0,0.75,12.0,-0.1387,-62.250000,1.0,40.000000,0
2,1,780,28,2.03,0,0,2.8679,0.81395,2.20,123,...,0,0,1,NaN,NaN,NaN,NaN,NaN,82.926826,0
3,2,780,43,2.09,1,0,2.8839,0.81395,1.60,123,...,0,0,1,-0.60,15.0,0.0160,-73.250000,1.0,82.926826,0
4,1,810,0,0.00,0,1,2.7634,0.88372,1.60,119,...,0,0,1,NaN,NaN,NaN,NaN,NaN,35.294117,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2,1170,76,2.21,0,0,2.8945,0.74627,2.80,177,...,0,0,1,0.63,12.0,0.2601,-207.799988,-1.0,29.943502,1
728,1,1010,27,2.44,1,0,2.7771,0.83636,2.20,322,...,0,0,1,NaN,NaN,NaN,NaN,NaN,52.173912,0
729,2,1010,42,2.35,1,0,3.1293,0.83636,2.50,322,...,0,0,1,0.30,15.0,0.3522,-81.100006,0.0,52.173912,0
730,1,700,0,0.00,1,1,2.6287,0.89189,2.50,200,...,1,0,0,NaN,NaN,NaN,NaN,NaN,40.000000,0


In [266]:
df.columns

Index(['term', 'sat', 'tothrs', 'cumgpa', 'season', 'frstsem', 'crsgpa',
       'verbmath', 'trmgpa', 'hssize', 'hsrank', 'id', 'spring', 'female',
       'black', 'white', 'ctrmgpa', 'ctothrs', 'ccrsgpa', 'ccrspop', 'cseason',
       'hsperc', 'football'],
      dtype='object')

In [267]:
df = df.loc[df['spring'] == 1]
x = df[['sat','tothrs','hsperc','female','black','white']]
y = df['cumgpa']
Regressao_Multipla(x,y,robusta='S')

                            OLS Regression Results                            
Dep. Variable:                 cumgpa   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.391
Method:                 Least Squares   F-statistic:                     39.30
Date:                Tue, 09 Mar 2021   Prob (F-statistic):           1.15e-36
Time:                        11:20:04   Log-Likelihood:                -238.90
No. Observations:                 366   AIC:                             491.8
Df Residuals:                     359   BIC:                             519.1
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4701      0.221      6.662      0.0

In [269]:
h0 = '(black = 0, white = 0)'
Teste_F(h0)

O valor de F é 0.733495 e seu p-valor é 0.480946. Portanto, não se rejeita Ho à significância de 5.0%, ou seja, as variáveis NÃO são conjuntamente significantes.


In [160]:
Resultado.wald_test(h0, cov_p=Resultado.cov_HC1).summary()

'<F test: F=array([[0.73349485]]), p=0.48094600268223997, df_denom=359, df_num=2>'

In [161]:
Resultado.wald_test(h0, cov_p=Resultado.cov_HC0).summary()

'<F test: F=array([[0.74779698]]), p=0.4741442714738484, df_denom=359, df_num=2>'